In [2]:
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, execute
import math
import random
import numpy as np
from scipy.optimize import minimize

In [10]:
import numpy as np
I = np.asmatrix('1 0;0 1')
X = np.asmatrix('0 1;1 0')
Y = np.asmatrix('0 0-1j;0+1j 0')
Z = np.asmatrix('1 0;0 -1')
pauli_i = [I, 0]
pauli_x = [X, 1]
pauli_y = [Y, 2]
pauli_z = [Z, 3]

def get_basis_set(num_of_qubit = 1):
    basis_set = [pauli_i, pauli_x, pauli_y, pauli_z]
    int_set = [pauli_i, pauli_x, pauli_y, pauli_z]
    temp_set = []
    for i in range (num_of_qubit-1):
        for j in range (len(basis_set)):
            for k in range (len(int_set)):
                a = basis_set[j]
                b = int_set[k]
                #temp_set.append()
                temp_set.append([np.kron(a[0],b[0]),[a[1],b[1]]])
        int_set = temp_set
        #print(int_set)
        temp_set = []
    return int_set

def get_decomposition(A, num_of_q):
    basis = get_basis_set(num_of_q)
    dec_list = []
    for i in range(len(basis)):
        b = basis[i]
        coeff = np.trace(A*b[0])/(2**num_of_q)
        dec_list.append([coeff, b[1]])
    return dec_list
def get_arbitary_decomposition(A):
    size = A.shape
    size = size[0]
    power = math.log2(size)
    p = int(power)
    if (p != power):
        p += 1
    ret = np.zeros([2**p,2**p], dtype = float)
    ret[:size, :size] = A
    for i in range(size, 2**p):
        ret[i,i] = 1
    return get_decomposition(ret, p)

In [31]:
backend = Aer.get_backend('qasm_simulator') 
T=8192
shots = T
max_credits = 3
def cal_cost(angle):
    den = 0 
    numer = 0
    qc = QuantumCircuit(2, 1)
    qc.h(0)
    qc.ry(angle[0], 1)
    qc.z(1)
    qc.ry(angle[1], 1)
    qc.cz(0, 1)
    qc.cz(0, 1)
    qc.h(0)
    qc.measure(0, 0)
    job = execute(qc, backend=backend, shots=shots,max_credits=max_credits)    
    result = job.result()
    counts = result.get_counts(qc)
    m_sum = 0
    if "0" in list (counts):
        m_sum=m_sum+counts["0"]/T
    if "1" in list (counts):
        m_sum=m_sum-counts["1"]/T
    den += 2*2*m_sum
    qc = QuantumCircuit(2, 1)
    qc.h(0)
    qc.ry(angle[0], 1)
    qc.z(1)
    qc.ry(angle[1], 1)
    qc.cz(0, 1)
    qc.cx(0, 1)
    qc.h(0)
    qc.measure(0, 0)
    job = execute(qc, backend=backend, shots=shots,max_credits=max_credits)    
    result = job.result()
    counts = result.get_counts(qc)
    m_sum = 0
    if "0" in list (counts):
        m_sum=m_sum+counts["0"]/T
    if "1" in list (counts):
        m_sum=m_sum-counts["1"]/T
    den += 2*0.5*m_sum
    qc = QuantumCircuit(2, 1)
    qc.h(0)
    qc.ry(angle[0], 1)
    qc.z(1)
    qc.ry(angle[1], 1)
    qc.cx(0, 1)
    qc.cz(0, 1)
    qc.h(0)
    qc.measure(0, 0)
    job = execute(qc, backend=backend, shots=shots,max_credits=max_credits)    
    result = job.result()
    counts = result.get_counts(qc)
    m_sum = 0
    if "0" in list (counts):
        m_sum=m_sum+counts["0"]/T
    if "1" in list (counts):
        m_sum=m_sum-counts["1"]/T
    den += 2*0.5*m_sum
    qc = QuantumCircuit(2, 1)
    qc.h(0)
    qc.ry(angle[0], 1)
    qc.z(1)
    qc.ry(angle[1], 1)
    qc.cx(0, 1)
    qc.cx(0, 1)
    qc.h(0)
    qc.measure(0, 0)
    job = execute(qc, backend=backend, shots=shots,max_credits=max_credits)    
    result = job.result()
    counts = result.get_counts(qc)
    m_sum = 0
    if "0" in list (counts):
        m_sum=m_sum+counts["0"]/T
    if "1" in list (counts):
        m_sum=m_sum-counts["1"]/T
    den += 0.5*0.5*m_sum
    
    qc = QuantumCircuit(3, 1)
    qc.h(0)
    qc.ry(angle[0], 1)
    qc.z(1)
    qc.ry(angle[1], 1)
    qc.cz(0, 1)
    qc.h(2)
    qc.cz(0, 2)
    qc.cx(1, 2)
    qc.h(0)
    qc.h(1)
    qc.measure(0, 0)
    job = execute(qc, backend=backend, shots=shots,max_credits=max_credits)    
    result = job.result()
    counts = result.get_counts(qc)
    m_sum = 0
    if "0" in list (counts):
        m_sum=m_sum+counts["0"]/T
    if "1" in list (counts):
        m_sum=m_sum-counts["1"]/T
    numer += 2*2*m_sum
    qc = QuantumCircuit(3, 1)
    qc.h(0)
    qc.ry(angle[0], 1)
    qc.z(1)
    qc.ry(angle[1], 1)
    qc.cz(0, 1)
    qc.h(2)
    qc.cx(0, 2)
    qc.cx(1, 2)
    qc.h(0)
    qc.h(1)
    qc.measure(0, 0)
    job = execute(qc, backend=backend, shots=shots,max_credits=max_credits)    
    result = job.result()
    counts = result.get_counts(qc)
    m_sum = 0
    if "0" in list (counts):
        m_sum=m_sum+counts["0"]/T
    if "1" in list (counts):
        m_sum=m_sum-counts["1"]/T
    numer += 2*0.5*m_sum
    qc = QuantumCircuit(3, 1)
    qc.h(0)
    qc.ry(angle[0], 1)
    qc.z(1)
    qc.ry(angle[1], 1)
    qc.cx(0, 1)
    qc.h(2)
    qc.cz(0, 2)
    qc.cx(1, 2)
    qc.h(0)
    qc.h(1)
    qc.measure(0, 0)
    job = execute(qc, backend=backend, shots=shots,max_credits=max_credits)    
    result = job.result()
    counts = result.get_counts(qc)
    m_sum = 0
    if "0" in list (counts):
        m_sum=m_sum+counts["0"]/T
    if "1" in list (counts):
        m_sum=m_sum-counts["1"]/T
    numer += 2*0.5*m_sum
    qc = QuantumCircuit(3, 1)
    qc.h(0)
    qc.ry(angle[0], 1)
    qc.z(1)
    qc.ry(angle[1], 1)
    qc.cx(0, 1)
    qc.h(2)
    qc.cx(0, 2)
    qc.cx(1, 2)
    qc.h(0)
    qc.h(1)
    qc.measure(0, 0)
    job = execute(qc, backend=backend, shots=shots,max_credits=max_credits)    
    result = job.result()
    counts = result.get_counts(qc)
    m_sum = 0
    if "0" in list (counts):
        m_sum=m_sum+counts["0"]/T
    if "1" in list (counts):
        m_sum=m_sum-counts["1"]/T
    numer += 0.5*0.5*m_sum
    cost = 1 - (numer/den)
    return abs(cost)

In [ ]:
angle = [0, 1.5714]
maxiter = 2000000
iters = 0 
threshold = 0.0000000047
k=0
out = minimize(cal_cost, x0=angle, method="cobyla", options={'maxiter':2000}, tol = threshold)
print(out)
qc = QuantumCircuit(1,1)
qc.ry(out.x[0], 0)
qc.z(0)
qc.ry(out.x[1], 0)
print(qc)
backend = Aer.get_backend('statevector_simulator')
job = execute(qc, backend)
result = job.result()
outputstate = result.get_statevector(qc, decimals=3)
print(outputstate)

In [17]:
I = np.asmatrix('1 0;0 1')
X = np.asmatrix('0 1;1 0')
Y = np.asmatrix('0 0-1j;0+1j 0')
Z = np.asmatrix('1 0;0 -1')
k = 2*Z + 0.5*X
b = np.asmatrix('0.7071;0.7071')
ans = np.linalg.inv(k)*b
print(ans)
print((ans)/np.linalg.norm(ans))

[[ 0.41594118]
 [-0.24956471]]
[[ 0.85749293]
 [-0.51449576]]


In [22]:
qc = QuantumCircuit(1,1)
qc.ry(1.58550022, 0)
qc.z(0)
qc.ry(2.00247779, 0)
backend = Aer.get_backend('statevector_simulator')
job = execute(qc, backend)
result = job.result()
outputstate = result.get_statevector(qc, decimals=3)
print(outputstate)

[0.978+0.j 0.207+0.j]


In [12]:
Ry1 = np.matrix([[np.cos(1.24011961/2.), -1*np.sin(1.24011961/2.)], [np.sin(1.24011961/2.), np.cos(1.24011961/2.)]])
Ry2 = np.matrix([[np.cos(1.87358632/2.), -1*np.sin(1.87358632/2.)], [np.sin(1.87358632/2.), np.cos(1.87358632/2.)]])
zKet = np.matrix('1;0')
V = Ry1*Z*Ry1
print (V*zKet)

[[1.00000000e+00]
 [4.91340772e-18]]


In [8]:
qc = QuantumCircuit(1,1)
qc.ry(0.314, 0)
qc.z(0)
print(qc)
backend = Aer.get_backend('statevector_simulator')
job = execute(qc, backend)
result = job.result()
outputstate = result.get_statevector(qc, decimals=3)
print(outputstate)

        ┌───────────┐┌───┐
q_0: |0>┤ Ry(0.314) ├┤ Z ├
        └───────────┘└───┘
 c_0: 0 ══════════════════
                          
[ 0.988+0.j -0.156+0.j]


In [13]:
Ry1 = np.matrix([[np.cos(0.314/2.), -1*np.sin(0.314/2.)], [np.sin(0.314/2.), np.cos(0.314/2.)]])
V = Ry1*Z
print (V*zKet)

[[0.98770079]
 [0.15635581]]


In [24]:
out.x

array([0.0732311 , 1.14861015])

In [54]:
backend = Aer.get_backend('qasm_simulator') 
T=100000
shots = T
max_credits = 3
qc = QuantumCircuit(2, 1)
qc.h(0)
qc.ry(np.pi/4, 1)
qc.z(1)
qc.ry(np.pi/2, 1)
qc.cx(0, 1)
#qc.cz(0, 1)
qc.h(0)
qc.measure(0, 0)
job = execute(qc, backend=backend, shots=T,max_credits=max_credits)    
result = job.result()
counts = result.get_counts(qc)
m_sum = 0
if "0" in list (counts):
    m_sum=m_sum+counts["0"]/T
if "1" in list (counts):
    m_sum=m_sum-counts["1"]/T
print(m_sum)
from qiskit.tools import visualization
print(counts)
#visualization.plot_histogram([counts["0"],counts["1"]])

0.0005599999999999494
{'0': 50028, '1': 49972}


In [48]:
Ry1 = np.matrix([[np.cos((np.pi/4)/2.), -1*np.sin((np.pi/4)/2.)], [np.sin((np.pi/4)/2.), np.cos((np.pi/4)/2.)]])
Ry2 = np.matrix([[np.cos((np.pi/2)/2.), -1*np.sin((np.pi/2)/2.)], [np.sin((np.pi/2)/2.), np.cos((np.pi/2)/2.)]])
V = Ry1*Z*Ry2
zKet = np.matrix('1;0')
si = V*zKet
si.T*X*si

matrix([[-0.70710678]])